In [1]:
import pandas as pd
import gc

In [2]:
new_column_names = [
    'beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date',
    'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall',
    'rating', 'text'
]

output_file_path = '../../minimizedData/RB_ratings_small.csv'

def parse_ratings_file(file_path, chunk_size=15_000_001, start=0, stop=200_000_000): #chunk is a mulitple of 17 since our text 'block' size is 17
    #initialise the csv
    pd.DataFrame(columns=new_column_names).drop(['user_name','abv','date','text'], axis=1).to_csv(
        output_file_path, index=False
    )

    current_block = []
    data = []
    i = 0
    k = 0 
    chunk = 0
    
    with open(file_path, 'r') as file:
        for line in file:
            i += 1
            k += 1
            if i<start: continue
           
            if i+16>stop:break
           
            if i % 10_000_000 == 0: print('Processing line: ', i)

            line = line.strip()  #remove whitespace

            if not line:  #empty line signifies end of block
                data.append(current_block)
                current_block = []
                
            if k >= chunk_size:
                k = 0
                chunk += 1
                print('Processing chunk :', chunk)
                
                df = pd.DataFrame(data, columns=new_column_names)
                df = df.drop(['user_name', 'abv', 'date', 'text'], axis=1)
                df.to_csv(output_file_path, mode='a', index=False, header=False)
                data = []  
                gc.collect()  #manually invoke garbage collection
                               
            if ': ' in line:  
                key, value = line.split(': ', 1)
                current_block.append(value)

        if data: #save any leftover data at the end
            data.append(current_block)
            current_block = []
            print(k)
            k = 0
            chunk += 1 
            print('Processing chunk :', chunk)
            
            df = pd.DataFrame(data, columns=new_column_names)
            df = df.drop(['user_name', 'abv', 'date', 'text'], axis=1)
            df.dropna()
            df.to_csv(output_file_path, mode='a', index=False, header=False)
            data = []  
            gc.collect() 
            
    print('Done!')

In [ ]:
parse_ratings_file('../../baseData/RateBeer/ratings.txt')

In [ ]:
df = pd.read_csv('../../minimizedData/RB_ratings_small.csv')
df = df.drop(df.index[-1])
df.to_csv('../../minimizedData/RB_ratings_small.csv', index=False)
df.tail(2)

,beer_name,beer_id,brewery_name,brewery_id,style,user_id,appearance,aroma,palate,taste,overall,rating
7122073,Svejk Dark,220898.0,Svejk Beer Garden,17155.0,Dunkel/Tmavý,10233.0,4.0,4.0,2.0,4.0,8.0,2.2
7122074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.head(2)

,beer_name,beer_id,brewery_name,brewery_id,style,user_id,appearance,aroma,palate,taste,overall,rating
0,33 Export (Gabon),410549.0,Sobraga,3198.0,Pale Lager,175852.0,2.0,4.0,2.0,4.0,8.0,2.0
1,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,442761.0,2.0,3.0,2.0,4.0,8.0,1.9


In [ ]:
df.sample(3)

,beer_name,beer_id,brewery_name,brewery_id,style,user_id,appearance,aroma,palate,taste,overall,rating
4289731,Blackstone Adam Bomb IPA,166792.0,Blackstone Restaurant & Brewery,2922.0,India Pale Ale (IPA),11056.0,4.0,5.0,4.0,7.0,14.0,3.4
4377333,Budweiser,473.0,Anheuser-Busch InBev,84.0,Pale Lager,68226.0,2.0,5.0,2.0,5.0,12.0,2.6
6051999,Richmond Swale,127820.0,Richmond,10523.0,Mild Ale,51287.0,4.0,6.0,2.0,5.0,10.0,2.7
